In [ ]:
!pip install openai

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import openai
import os
import pandas as pd
import time
import csv
import json
import requests
import re

In [ ]:
# Load the configuration file
with open('drive/My Drive/Data/Config/OpenAI_config.json') as config_file:
    config = json.load(config_file)

In [ ]:
dataframe = pd.read_csv('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Sampled_Posts_1000.csv')

In [ ]:
dataframe=dataframe.astype(str)
dataframe = dataframe.replace('\n','', regex=True)

In [ ]:
dataframe

,postID,post,word_count,category
0,xapb9j,I forgot to take my pill yesterday. Is it okay...,51,Group 1
1,va1s9b,how do i lose weight?. i’ve gained a lot of we...,84,Group 1
2,vyv2ut,Lab results. I just got my labs back and I’d l...,45,Group 1
3,wy5f9k,Can you get rid of the follicles/cysts?. No ot...,61,Group 1
4,yvjy63,hair loss. i’m slowly balding from my scalp. i...,43,Group 1
...,...,...,...,...
995,y87em5,PCOS Beginner Questions. Hi all! I don’t have ...,344,Group 4
996,v96ws6,Insulin Resistance/Sugar Readings. Hello! I wa...,353,Group 4
997,yj6zyi,Weird coincidence??. So I have no idea what’s ...,301,Group 4
998,y6k0n6,Hormone irregularity. I’ll try not to make thi...,346,Group 4


In [ ]:
df_corpus = pd.DataFrame()
df_corpus['postID']=dataframe['postID']
df_corpus['post'] = dataframe['post']

In [ ]:
df_corpus['postID'][0]

'xapb9j'

In [ ]:
from openai import OpenAI
api_key = config['OPENAI_API_KEY']

In [ ]:
client = OpenAI(
     api_key=api_key,
     max_retries=2,
     timeout=180.0,
)
openai_query_params = {"model": "gpt-4o", "temperature": 0.25, "max_tokens": 4096}

def ask_openai(prompt: str, openai_query_params=openai_query_params):
    messages = [
        {"role": "system", "content": "You are an expert in medical information extraction."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
          messages=messages,
        **openai_query_params,
    )
    return response.choices[0].message.content



In [ ]:
zs_short_base_prompt="""
###Task###
Identify the physical symptoms of Polycystic Ovarian Syndrome (PCOS) mentioned by the narrator in the given post.

###Instructions###
**Include symptoms that are caused due to PCOS (implicit or explicit mention of PCOS).
**Exclude any psychological symptoms.
**List the symptoms separated by commas, or return 'none' if no symptoms identified.
"""

In [ ]:
responses= []

#Generating Responses

In [ ]:
# Iterate over the posts
for i in range(750,1000):
    postID = df_corpus['postID'][i]
    post = df_corpus['post'][i]
    prompt = zs_short_base_prompt + "\n" + "Post:" + post
    api_response = ask_openai(prompt)
    # Append the valid response
    responses.append({"postID":postID, "post":post, "Symptoms Found(LLM)": api_response})
# Convert the list of responses to a pandas DataFrame
df_responses = pd.DataFrame(responses)

In [ ]:
df_responses

In [ ]:
df_responses.to_excel('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Dataset (Baseline Comparisons)/PCOS Symptoms (ChatGPT-4o).xlsx', index=False)